In [ ]:
#currency exchange.py

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.pylab as pylab
import matplotlib.ticker as mticks
from datetime import datetime
import time
import requests
from requests.exceptions import HTTPError
import json
fig, ax = plt.subplots(1,1)
ax.set_ylim([0,2.0]) 
x = []
y = []
width = 5
range = 0.5
max_curr = min_curr = 0 


def animate(i):
    # fetch the exchange rates
    try:
        url = "http://data.fixer.io/api/latest?access_key=" + \
                   "46a0a81c015f1e2bda5059645ef1282a&format=1"
        #remembe to replace the access keys 
        
        response = requests.get(url)
        # Raise an exception if a request is unsuccessful
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  
    except Exception as err:
        print(f'Other error occurred: {err}')  
    else:
        try:
            rates = json.loads(response.text)    
        except ValueError as err:  
            print(err)
        else:        
            curr = float(rates["rates"]["SGD"])
    print(curr)
    global max_curr
    global min_curr
    # save the max and of the curr
    max_curr  = max(curr, max_curr)
    min_curr = max_curr if min_curr == 0 else min(curr, min_curr)

    # add the data for x and y-axis
    now = datetime.now() 
    x.append(now.strftime("%H:%M:%S"))
    y.append(curr)
    
    # clear the chart and plot again
    ax.clear()
    ax.plot(x, y)
    if i > width:
        ax.set(xlim=(i - width, i+1))  # shift the x-axis labels
    
    # set the y-limit
    ax.set_ylim(( min(curr-range/2, min_curr), max(curr+range/2, max_curr))) 
    ax.yaxis.set_major_locator(       # display the y-axis label with intervals 
                                      # of 0.05 
        mticks.MultipleLocator(0.05)) # no. of labels = (range)/0.05
    
    ax.fill_between(
        x, y, 0, 
        facecolor='yellow', 
        alpha=0.5)    
    locs, _ = plt.yticks()       # get the ticks and their corresponding labels    
    plt.yticks(locs, map(        # format the labels to display 5 decimal places
        lambda x: "%.5f" % x, 
        locs))   
ani = animation.FuncAnimation(fig, animate, interval=2000)
plt.show()
# Press Q to quit